# MEx 1: The Julia framework for HPC
Johnrob Y. Bantang, Ph.D.

## **OBJECTIVE**: Explore Julia framework for HPC
- [ ] KR1: Ran different versions of `@code_[mode]` to examine a simple expression (replace `[mode]` with `native`, `typed`, `warntype`, etc.
- [ ] KR2: Showed that constants in codes are passed on after constant function or expression evaluations. May have gone beyond replicating those done in class.
- [ ] KR3: Created two versions of the same function such as `pos(x)` in the book, one with type-instability and the other fixed.
- [ ] KR4: Demonstrated Julia's type-inference and multiple dispatch using the `iam()` function sample as discussed in class. (See Chapter 3)
- [ ] KR5: Ran a function `poly(A,x)` that evaluates the $N$-th order polynomial via a naive approach.
- [ ] KR6: Evaluated the same polynomial using [the package `Polynomials` in Julia](https://juliamath.github.io/Polynomials.jl/stable/).
- [ ] KR7. Used `BenchmarkTools` to plot timing distribution and obtain the best time via the macro `@btime` evaluated at some random number. (See Chapter 2)
- [ ] KR8 (Bonus): Plotted of the time $T$ it takes for the function to run using `@time` macro for different polynomial lengths $N$.

[Full documentation is found here.](./README.md)

## Type hierarchy

Type hierarchy in Julia is a tree. With `Any` on top of it. All numbers can be arranged using the recursive call of a function found in [a wikibook](https://en.wikibooks.org/wiki/Introducing_Julia/Types).

In [1]:
function showsubtree( T, level=0 )
    println("  "^level, T)
    for t in subtypes(T)
        showsubtree(t, level+1)
    end
end

showsubtree (generic function with 2 methods)

In [2]:
showsubtree(Number)

Number
  Base.MultiplicativeInverses.MultiplicativeInverse
    Base.MultiplicativeInverses.SignedMultiplicativeInverse
    Base.MultiplicativeInverses.UnsignedMultiplicativeInverse
  Complex
  Real
    AbstractFloat
      BigFloat
      Float16
      Float32
      Float64
    AbstractIrrational
      Irrational
    Integer
      Bool
      Signed
        BigInt
        Int128
        Int16
        Int32
        Int64
        Int8
      Unsigned
        UInt128
        UInt16
        UInt32
        UInt64
        UInt8
    Rational


## Memory address and value

In programming, variables are symbols that _point_ to a memory location and therefore they contain a memory address. Their values are obtained as a reading of the site state according to the appropriate size and/or stride size, with directions and sequence peculiar to a given implementation, i.e. programming language.

There are two equality tests in Julia: address equality (physical, [===, also ≡ (`\equiv`)](https://docs.julialang.org/en/v1/base/base/#Core.:===)) and value equality (logical, [==](https://docs.julialang.org/en/v1/base/math/#Base.:==)). There is also a related and similar native function [`isequal()`](https://docs.julialang.org/en/v1/base/base/#Base.isequal).

## Memory architecture

Due to differences in the physical layout and design (physical and relational) according to manufacturers, actual speeds may vary between specific machines and across models.
The most common architecture is [the x86-64, the 64-bit version of the x86 instruction set](https://en.wikipedia.org/wiki/X86-64).

### Hence:
Specify machine architecture in all numerical work.

## Equality and equivalence

- Recommended _model_:
    - equality is _by value_ and
    - equivalence is by _memory location or reference_.

- In `c/c++` functions either do _value-passing_ or _reference-passing_ between functions.

- Obvious difference: variables beyond the native fundamental types
    - `struct`
    - other structures and containers (`Array`s).

## Case in `Point`

In [3]:
struct Point
    x::Int
    y::Int
end

Point() = Point(0,1) #Default object constructor

Point

**QUESTION:** Why not use a two-element `Vector` instead?

## Structure elements are references

- References are _by default_ fixed (constant).
- However: References pointed to by those elements **may** be references themselves whose values can be modified.

In [4]:
pt1 = Point();
pt2 = Point();

@show pt1
@show pt2

@show pt1 == pt2
@show pt1 === pt2
@show pt1.x === pt2.x
@show pt1.y === pt2.y;

pt1 = Point(0, 1)
pt2 = Point(0, 1)
pt1 == pt2 = true
pt1 === pt2 = true
pt1.x === pt2.x = true
pt1.y === pt2.y = true


In [5]:
pt3 = Point(1,1)
@show pt3;

pt3 = Point(1, 1)


In [6]:
@show pt2 === pt3;

pt2 === pt3 = false


## Default: immutable struct
While elements are accessible ("exposed" or "public"), they are not mutable.

In [7]:
@show pt4

println("Changing x-value of pt4...")
pt4.x = 0

LoadError: UndefVarError: `pt4` not defined

## LLVM

- Low-level generator of code from high-level codes (Low Level Virtual Machine)
- Dependent on:
    1. Multiple dispatch, and
    1. Type stability

## Default inlining

Consider the two functions
$$f(x) = 5x + 3$$
and
$$g(x) = f(2x)$$

In [47]:
f(x) = 5x + 3
g(x) = f(2x)

g (generic function with 1 method)

In [48]:
@code_typed g(3)

CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = Base.mul_int(5, %1)::Int64
│   %3 = Base.add_int(%2, 3)::Int64
└──      return %3
) => Int64

## Inlined result

Note that
$$g(x) = f(2x) = 5(2x) + 3 = 10x + 3$$
and the LLVM processed code now looks as follows.

In [51]:
@code_llvm g(3)

;  @ In[47]:2 within `g`
define i64 @julia_g_4193(i64 signext %0) #0 {
top:
; ┌ @ In[47]:1 within `f`
; │┌ @ int.jl:88 within `*`
    %1 = mul i64 %0, 10
; │└
; │┌ @ int.jl:87 within `+`
    %2 = add i64 %1, 3
; └└
  ret i64 %2
}


The inlined multiplication $(5 \cdot 2)x$ are combined to form $10x$ within the function.

## Method (function)

Mapping of the input `Tuple` to the output `Tuple`

## Multiple dispatch

- Unique machine codeblocks for each unique mapping ("method")
- Similar to polymorphism in other programming languages

In [8]:
? ^(x::Number,y::Number)

```
^(x, y)
```

Exponentiation operator. If `x` is a matrix, computes matrix exponentiation.

If `y` is an `Int` literal (e.g. `2` in `x^2` or `-3` in `x^-3`), the Julia code `x^y` is transformed by the compiler to `Base.literal_pow(^, x, Val(y))`, to enable compile-time specialization on the value of the exponent. (As a default fallback we have `Base.literal_pow(^, x, Val(y)) = ^(x,y)`, where usually `^ == Base.^` unless `^` has been defined in the calling namespace.) If `y` is a negative integer literal, then `Base.literal_pow` transforms the operation to `inv(x)^-y` by default, where `-y` is positive.

# Examples

```jldoctest
julia> 3^5
243

julia> A = [1 2; 3 4]
2×2 Matrix{Int64}:
 1  2
 3  4

julia> A^3
2×2 Matrix{Int64}:
 37   54
 81  118
```


In [9]:
@code_typed 3^5

CodeInfo(
1 ─ %1 = invoke Base.power_by_squaring(x::Int64, p::Int64)::Int64
└──      return %1
) => Int64

In [10]:
@code_typed 3^5.0

CodeInfo(
1 ─ %1 = Base.sitofp(Float64, x)::Float64
│   %2 = invoke Base.:^(%1::Float64, y::Float64)::Float64
└──      return %2
) => Float64

In [11]:
"3"^5

"33333"

In [12]:
@code_typed "3"^5

CodeInfo(
1 ─ %1 = invoke Base.repeat(s::String, r::Int64)::String
└──      return %1
) => String

In [13]:
methods(^)

# 68 methods for generic function "^" from Base:
  [1] ^(x::Bool, y::BigInt)
     @ Base.GMP gmp.jl:656
  [2] ^(x::Bool, y::Bool)
     @ bool.jl:169
  [3] ^(r::Regex, i::Integer)
     @ regex.jl:863
  [4] ^(x::Float16, y::Integer)
     @ Base.Math math.jl:1283
  [5] ^(::Irrational{:ℯ}, A::AbstractMatrix)
     @ LinearAlgebra ~/.julia/juliaup/julia-1.10.1+0.x64.apple.darwin14/share/julia/stdlib/v1.10/LinearAlgebra/src/dense.jl:645
  [6] ^(::Irrational{:ℯ}, x::Complex)
     @ Base.MathConstants mathconstants.jl:119
  [7] ^(::Irrational{:ℯ}, x::Integer)
     @ Base.MathConstants mathconstants.jl:119
  [8] ^(::Irrational{:ℯ}, x::Rational)
     @ Base.MathConstants mathconstants.jl:119
  [9] ^(::Irrational{:ℯ}, x::AbstractIrrational)
     @ Base.MathConstants mathconstants.jl:119
 [10] ^(x::Float32, n::Integer)
     @ Base.Math math.jl:1277
 [11] ^(x::Float64, n::Integer)
     @ Base.Math math.jl:1246
 [12] ^(x::BigInt, y::Bool)
     @ Base.GMP gmp.jl:653
 [13] ^(x::BigInt, y::BigInt)
     @ Base.GMP gmp.jl:652
 [14] ^(x::BigInt, y::UInt64)
     @ Base.GMP gmp.jl:628
 [15] ^(x::BigInt, y::Integer)
     @ Base.GMP gmp.jl:654
 [16] ^(x::Integer, y::BigInt)
     @ Base.GMP gmp.jl:655
 [17] ^(x::Integer, y::Bool)
     @ bool.jl:170
 [18] ^(x::BigFloat, y::BigInt)
     @ Base.MPFR mpfr.jl:664
 [19] ^(x::BigFloat, y::Union{UInt16, UInt32, UInt64, UInt8})
     @ Base.MPFR mpfr.jl:652
 [20] ^(x::BigFloat, y::Unsigned)
     @ Base.MPFR mpfr.jl:671
 [21] ^(x::BigFloat, y::Union{Int16, Int32, Int64, Int8})
     @ Base.MPFR mpfr.jl:658
 [22] ^(x::BigFloat, y::Integer)
     @ Base.MPFR mpfr.jl:670
 [23] ^(x::T, p::T) where T<:Integer
     @ intfuncs.jl:310
 [24] ^(x::Rational, n::Integer)
     @ rational.jl:507
 [25] ^(z::Complex{<:Rational}, n::Bool)
     @ rational.jl:515
 [26] ^(z::Complex{<:AbstractFloat}, n::Bool)
     @ complex.jl:869
 [27] ^(z::Complex{<:Integer}, n::Bool)
     @ complex.jl:870
 [28] ^(z::Complex{<:Rational}, n::Integer)
     @ rational.jl:516
 [29] ^(z::Complex, n::Bool)
     @ complex.jl:866
 [30] ^(z::Complex{<:Integer}, n::Integer)
     @ complex.jl:874
 [31] ^(z::Complex{<:AbstractFloat}, n::Integer)
     @ complex.jl:872
 [32] ^(z::Complex, n::Integer)
     @ complex.jl:867
 [33] ^(x::T, y::Rational) where T<:AbstractFloat
     @ rational.jl:512
 [34] ^(z::Complex{T}, p::Rational) where T<:Real
     @ rational.jl:513
 [35] ^(::Irrational{:ℯ}, x::Number)
     @ Base.MathConstants mathconstants.jl:119
 [36] ^(z::T, p::Complex{S}) where {T<:Real, S<:Real}
     @ complex.jl:880
 [37] ^(z::T, p::Complex{T}) where T<:Real
     @ complex.jl:864
 [38] ^(x::Number, p::Integer)
     @ intfuncs.jl:311
 [39] ^(z::Complex{T}, p::S) where {T<:Real, S<:Real}
     @ complex.jl:876
 [40] ^(x::Number, y::Rational)
     @ rational.jl:511
 [41] ^(z::Complex{T}, p::T) where T<:Real
     @ complex.jl:863
 [42] ^(x::AbstractIrrational, y::AbstractIrrational)
     @ irrationals.jl:161
 [43] ^(x::Float64, y::Float64)
     @ Base.Math math.jl:1194
 [44] ^(::Missing, ::Missing)
     @ missing.jl:122
 [45] ^(::Missing, ::Integer)
     @ missing.jl:165
 [46] ^(::Missing, ::Number)
     @ missing.jl:123
 [47] ^(x::BigFloat, y::BigFloat)
     @ Base.MPFR mpfr.jl:646
 [48] ^(b::Number, A::AbstractMatrix)
     @ LinearAlgebra ~/.julia/juliaup/julia-1.10.1+0.x64.apple.darwin14/share/julia/stdlib/v1.10/LinearAlgebra/src/dense.jl:643
 [49] ^(::Number, ::Missing)
     @ missing.jl:124
 [50] ^(z::Complex{T}, p::Complex{T}) where T<:Real
     @ complex.jl:862
 [51] ^(x::T, y::T) where T<:Union{Float16, Float32}
     @ Base.Math math.jl:1224
 [52] ^(x::T, y::T) where T<:Number
     @ promotion.jl:511
 [53] ^(x::Number, y::Number)
     @ promotion.jl:456
 [54] ^(A::LinearAlgebra.Hermitian, p::Integer)
     @ LinearAlgebra ~/.julia/juliaup/julia-1.10.1+0.x64.apple.darwin14/share/julia/stdlib/v1.10/LinearAlgebra/src/symmetric.jl:652
 [55] ^(D::LinearAlgebra.Diagonal, a::Integer)
     @ LinearAlgebra ~/.julia/juliaup/julia-1.10.1+0.x64.apple.darwin14/share/j

In [14]:
@code_llvm 3^5

;  @ intfuncs.jl:310 within `^`
define i64 @"julia_^_1896"(i64 signext %0, i64 signext %1) #0 {
top:
  %2 = call i64 @j_power_by_squaring_1898(i64 signext %0, i64 signext %1)
  ret i64 %2
}


In [15]:
@code_lowered 3^5

CodeInfo(
1 ─ %1 = Base.power_by_squaring(x, p)
└──      return %1
)

In [16]:
@code_native 3^5

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 14, 0
	.globl	"_julia_^_1997"                 ## -- Begin function julia_^_1997
	.p2align	4, 0x90
"_julia_^_1997":                        ## @"julia_^_1997"
; ┌ @ intfuncs.jl:310 within `^`
## %bb.0:                               ## %top
	sub	rsp, 8
	movabs	rax, offset _j_power_by_squaring_1999
	call	rax
	pop	rcx
	ret
; └
                                        ## -- End function
.subsections_via_symbols


In [17]:
@code_typed 3^5

CodeInfo(
1 ─ %1 = invoke Base.power_by_squaring(x::Int64, p::Int64)::Int64
└──      return %1
) => Int64

In [18]:
@code_warntype 3^5

MethodInstance for ^(::Int64, ::Int64)
  from ^(x::T, p::T) where T<:Integer @ Base intfuncs.jl:310
Static Parameters
  T = Int64
Arguments
  #self#::Core.Const(^)
  x::Int64
  p::Int64
Body::Int64
1 ─ %1 = Base.power_by_squaring(x, p)::Int64
└──      return %1



## Own multiple dispatch

In [19]:
iswhat(x::Number) = " is a number."
iswhat(x::String) = " is a string."
iswhat(x::Int) = " is an integer."
iswhat(x::Rational{Int}) = " is a rational number."
iswhat(x::Complex) = " is a complex number."

iswhat (generic function with 5 methods)

In [20]:
println( 5, iswhat(5) );
println( 5.0, iswhat(5.0) );
println( 3//5, iswhat(3//5) );
println( pi, iswhat(pi) );
println( (3+5im)^2, iswhat((3+5im)^2) );

5 is an integer.
5.0 is a number.
3//5 is a rational number.
π is a number.
-16 + 30im is a complex number.


## Type inference

- Type information
    - memory block size
    - expansion schedule (i.e. containers)
- Not necessary to specify types in Julia like other high-level languages

### A vector of squared integers

In [21]:
[ x^2 for x in 0:10 ]

11-element Vector{Int64}:
   0
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

### A vector of squared floats

In [22]:
[ x^2 for x in 0.0:10.0 ]

11-element Vector{Float64}:
   0.0
   1.0
   4.0
   9.0
  16.0
  25.0
  36.0
  49.0
  64.0
  81.0
 100.0

## Type stability

- Type information
    - memory block size
    - expansion schedule (i.e. containers)
- Not necessary to specify types in Julia like other high-level languages
- Inference: type information determined at runtime
    - Julia: best-effort
    - OCaml, F#: required
- Stability: deterministic flow of type information in all processes
    - Functions must return the same type for every given tuple of types as input
    - Multiple dispatch

In [23]:
? ramp(x)

No documentation found.

Binding `ramp` does not exist.


In [24]:
ramp(x) = (x ≤ 0) ? 0 : x

ramp (generic function with 1 method)

In [25]:
@show ramp(1.0);
@show ramp(1);
@show ramp(-1.0);

ramp(1.0) = 1.0
ramp(1) = 1
ramp(-1.0) = 0


In [26]:
@code_warntype ramp(1)

MethodInstance for ramp(::Int64)
  from ramp(x) @ Main In[24]:1
Arguments
  #self#::Core.Const(ramp)
  x::Int64
Body::Int64
1 ─ %1 = (x ≤ 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x



In [27]:
@code_warntype ramp(1.0)

MethodInstance for ramp(::Float64)
  from ramp(x) @ Main In[24]:1
Arguments
  #self#::Core.Const(ramp)
  x::Float64
Body::Union{Float64, Int64}
1 ─ %1 = (x ≤ 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x



In [28]:
using Pkg;
Pkg.activate(".");
Pkg.add("BenchmarkTools");

using BenchmarkTools;

  Activating project at `~/Documents/GitHub/Phys215-202324-2/01-HPC`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Phys215-202324-2/01-HPC/Project.toml`
  No Changes to `~/Documents/GitHub/Phys215-202324-2/01-HPC/Manifest.toml`


In [29]:
x = randn(1_000);
trial1 = @benchmark ramp.($x)

BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):   6.436 μs …  1.425 ms  ┊ GC (min … max):  0.00% … 98.57%
 Time  (median):      7.985 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   10.732 μs ± 48.157 μs  ┊ GC (mean ± σ):  21.89% ±  4.99%

   ▂▄▅▇██████▇▇▆▆▄▄▂▂               ▁ ▂ ▁▁▁▁                  ▃
  ▆██████████████████████▇▆▇▆▇▇▇▇█▇██████████▇███▆▆▆▅▅▅▅▅▄▄▅▄ █
  6.44 μs      Histogram: log(frequency) by time      16.2 μs <

 Memory estimate: 23.44 KiB, allocs estimate: 486.

In [30]:
x = rand(-100:100, 1_000)
trial0 = @benchmark ramp.($x)

BenchmarkTools.Trial: 9504 samples with 202 evaluations.
 Range (min … max):  547.069 ns … 82.807 μs  ┊ GC (min … max):  0.00% … 98.26%
 Time  (median):       1.109 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.582 μs ±  7.324 μs  ┊ GC (mean ± σ):  53.13% ± 17.94%

  █▆▂▁▂                                                        ▁
  ██████▇▅▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▄▁▃▄▅▄▄▄▃▅▄▄▅▆▅▄▆▆▇▆▆▇▆▆▆▆▆▇▆ █
  547 ns        Histogram: log(frequency) by time      42.8 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

## Relative speed difference

In [31]:
? speedup

search:

Couldn't find speedup
Perhaps you meant sleep or step


No documentation found.

Binding `speedup` does not exist.


In [32]:
function speedup(t₀::BenchmarkTools.Trial, t₁::BenchmarkTools.Trial)
    return median(t₁.times) / median(t₀.times)
end

speedup (generic function with 1 method)

In [33]:
x = randn(1_000);
trial1 = @benchmark ramp.($x);

x = rand(-100:100, 1_000);
trial0 = @benchmark ramp.($x);

In [34]:
su = speedup(trial0,trial1)
println("Trial 1 / Trial 0 ≈ ", round( su, digits=3 ) )
println( "Trial 1 is ", (su > 1) ? "SLOWER" : "FASTER", " than Trial 0." )

Trial 1 / Trial 0 ≈ 7.484
Trial 1 is SLOWER than Trial 0.


## Fixed `ramp()`

In [35]:
ramp(x) = (x ≤ 0) ? zero(x) : x

ramp (generic function with 1 method)

In [36]:
x = randn(1_000);
trial1 = @benchmark ramp.($x);

x = rand(-100:100, 1_000);
trial0 = @benchmark ramp.($x);

In [37]:
su = speedup(trial0,trial1)
println("Trial 1 / Trial 0 ≈ ", round( su, digits=3 ) )
println( "Trial 1 is ", (su < 1) ? "SLOWER" : "FASTER", " than Trial 0." )

Trial 1 / Trial 0 ≈ 1.015
Trial 1 is FASTER than Trial 0.


## Modularity: method (function) instead of script

The use of **global variables** are a boon and highly discouraged in Julia. This forces users to think of modular approach in tasking.

Case in point. An apparent $20\times$ speed difference between MATLAB and Julia has been flagged in [a Julia discourse](https://discourse.julialang.org/t/matlab-outperforms-julia-20-times-faster-running-this-nested-loop/92691/3). It turns out that modularization alone (pure local variables, within functions) can reverse the speed difference to Julia instead being $30\times$ faster than MATLAB (to be balanced, the posts did not mention if the second run-time difference is repeated). This reversal means that an effective $600\times$ enhancement in the Julia code can be achieved by modularization alone.

## Polynomial evaluator

- Input:
    - $a_n$, $n=0,1,\ldots,N$
    - $x$, the variable to evaluate $p(x)$
    - The polynomial degree $N$ is encoded in the length $|a|$.
- Output:
    - value of $p(x)$ (scalar)

## Naive implementation

- Naive `for` loop over each coefficient `A[n]` multiplying each with the input variable `x` raised to the corresponding power.
- Set $a_0$ as the first element `A[1]`.
- Final formula, given $A_n$, with $n=1,2,\ldots,N+1$ (shifted)
$$p(x) = \sum_{n=1}^{N+1} A_n x^{n-1}$$

## Naive implementation code

In [38]:
function poly(A,x)
    p = zero(x) #ensure type stability
    n = 0
    for a in A
        p += a * x^n
        n += 1
    end
    return p
end

poly (generic function with 1 method)

## Use random coefficients

In [39]:
N = 100;
A = rand(N+1);
x = rand();

In [40]:
@show poly(A,x);

poly(A, x) = 0.49001280754888865


In [41]:
@benchmark poly($A,$x)

BenchmarkTools.Trial: 10000 samples with 50 evaluations.
 Range (min … max):  942.080 ns …  11.126 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):       1.419 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.460 μs ± 555.940 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆▅▆█▄▄▂▄▃ ▃▅ ▇ █▃▃▂▁▁   ▁  ▁       ▁     ▁                    ▂
  █████████▇██████████████████████████▇██████████▇▆▆▆▆▆▆▆▆▆▆▆▅▆ █
  942 ns        Histogram: log(frequency) by time        3.4 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

## Using `Polynomials.jl`

In [42]:
Pkg.add("Polynomials")

using Polynomials

   Resolving package versions...
  No Changes to `~/Documents/GitHub/Phys215-202324-2/01-HPC/Project.toml`
  No Changes to `~/Documents/GitHub/Phys215-202324-2/01-HPC/Manifest.toml`


In [43]:
p = Polynomial(A)

Polynomial(0.40085889775727346 + 0.00462951851918203*x + 0.8629002922122636*x^2 + 0.28990843849057346*x^3 + 0.19010881046758743*x^4 + 0.5751990292995522*x^5 + 0.962028813380673*x^6 + 0.7538804802027133*x^7 + 0.6192700835383871*x^8 + 0.32434894094277855*x^9 + 0.2622300738220563*x^10 + 0.1046944071665642*x^11 + 0.011753638250217913*x^12 + 0.5064966771810419*x^13 + 0.3767640273291225*x^14 + 0.12362806640148849*x^15 + 0.47270709446242287*x^16 + 0.6487956309287983*x^17 + 0.6615437825239098*x^18 + 0.20333065927105076*x^19 + 0.839445961674062*x^20 + 0.6905076902287091*x^21 + 0.31455977307744787*x^22 + 0.6062532727736188*x^23 + 0.3195686881040888*x^24 + 0.254626272707944*x^25 + 0.07751903450456277*x^26 + 0.3062384402929783*x^27 + 0.7150670958234514*x^28 + 0.4973413963269121*x^29 + 0.9702688522126935*x^30 + 0.8660489339427895*x^31 + 0.5929711634452731*x^32 + 0.5183565795969004*x^33 + 0.4876972991145839*x^34 + 0.6420667683413701*x^35 + 0.2325895714061751*x^36 + 0.36762742512614*x^37 + 0.7289607718990039*x^38 + 0.38013068535882355*x^39 + 0.5825776947287318*x^40 + 0.5610474920143527*x^41 + 0.27655360454207567*x^42 + 0.84399173179996*x^43 + 0.8831987982121432*x^44 + 0.679334945730744*x^45 + 0.40770094740990515*x^46 + 0.9015712966059968*x^47 + 0.8730002892045262*x^48 + 0.12258271476536309*x^49 + 0.7924676287393794*x^50 + 0.10205297845083794*x^51 + 0.15363647273396352*x^52 + 0.8596220587784603*x^53 + 0.8005082569418368*x^54 + 0.23242362234498504*x^55 + 0.9270079473640556*x^56 + 0.9455376763945134*x^57 + 0.475249657195679*x^58 + 0.2836792409009613*x^59 + 0.7272230541088214*x^60 + 0.5873242085726189*x^61 + 0.8983175954980213*x^62 + 0.656813476415098*x^63 + 0.7034989439097769*x^64 + 0.0032479931054700817*x^65 + 0.7921912185388008*x^66 + 0.7320216627289011*x^67 + 0.15957224265259407*x^68 + 0.6353748007585084*x^69 + 0.3310486716411618*x^70 + 0.4019040223164706*x^71 + 0.5014983029709666*x^72 + 0.4084853975716579*x^73 + 0.7399485980761216*x^74 + 0.7249024912001755*x^75 + 0.006270505245247304*x^76 + 0.028008375917608408*x^77 + 0.20880490893827552*x^78 + 0.4042792916604999*x^79 + 0.9283683858435997*x^80 + 0.5661716381405258*x^81 + 0.38623054639559806*x^82 + 0.25610583016010857*x^83 + 0.10138667938448909*x^84 + 0.1635768157368036*x^85 + 0.653865484585075*x^86 + 0.13547567593158116*x^87 + 0.5138250944244783*x^88 + 0.08030753113237665*x^89 + 0.8028723493483443*x^90 + 0.6028848482410909*x^91 + 0.36476029167849644*x^92 + 0.47521244024073894*x^93 + 0.9905014549179411*x^94 + 0.9153938340357425*x^95 + 0.16870109961901059*x^96 + 0.8016190369940805*x^97 + 0.5311702698666498*x^98 + 0.7746249868482967*x^99 + 0.6850540588438154*x^100)

## Evaluate relative speed

In [44]:
N = 100;
A = rand(N+1);
x = rand();

p = Polynomial(A);

@show poly(A,x);
@show p(x);

poly(A, x) = 3.962827456140382
p(x) = 3.9628274561403787


In [45]:
trial0 = @benchmark poly($A,$x);
trial1 = @benchmark p($x);

In [46]:
su = speedup(trial0,trial1)
println("Trial 1 / Trial 0 ≈ ", round( su, digits=3 ) )
println( "Trial 1 is ", (su > 1) ? "SLOWER" : "FASTER", " than Trial 0." )

Trial 1 / Trial 0 ≈ 0.138
Trial 1 is FASTER than Trial 0.


## Fin.
1. Julia provides macros for examining machine code generation.
2. Modularity via methods (or functions) provide opportunity for optimization.
3. Use benchmarking tools whenever available to examine relative speeds.
4. Existing optimized modules may be worth considering before using naive implementations.

## Explore.